In [1]:
import pandas as pd
import math
import numpy as np
#%matplotlib ipympl
import matplotlib.pyplot as plt

In [ ]:
# #Datasets to read from 
# rawdata = [0]
# rawdata(1) = "B_N_10"
# rawdata(2) = "B_N_12"
# rawdata(3) = "B_U_9"
# rawdata(4) = "B_U_11"
# rawdata(5) = "DB_N_5"
# rawdata(6) = "DB_N_7"
# rawdata(7) = "DB_U_6"
# rawdata(8) = "DB_U_8"


read .csv and reformat as dataframe with absolute force and displacement values 

In [ ]:
BN12 = pd.read_csv(r"C:\Users\arjun\Documents\2K\B_N_12.txt", sep='\t', header=[5])
BN12 = BN12.drop(BN12.columns[[2,3]],axis=1).abs()

BN12_new = slopefix(BN12,0.5,0.7)

In [ ]:
DB_N_7 = pd.read_csv(r"C:\Users\arjun\Documents\2K\DB_N_7.txt", sep='\t', header=[5])
DB_N_7.plot(x='mm',y='N')

In [3]:
DB_N_5 = pd.read_csv(r"C:\Users\arjun\Documents\2K\DB_N_5.txt", sep='\t', header=[5])
DB_N_5 = DB_N_5.drop(DB_N_5.columns[[2,3]],axis=1).abs()

dataset = DB_N_5
# DB_N_5_new = slopefix(dataset,0.5,0.9)

In [48]:
# function to crop based on cutoff limit

def cropdata(dataset,n)

    #dataset = the specimen data that you want cleaned
    #n = %cutoff, ie. 0.1 = 90% of postpeak
    #return truncated dataset removing everything after 'n'

    max_val = dataset['N'].max() #initialize
    max_index = dataset['N'].idxmax() #initialize

    dataset_temp = dataset.truncate(before=max_index) #.reset_index(drop=True)

    cutoff = dataset_temp.iloc[(dataset_temp['N']-(max_val*n)).abs().argsort()[:1]]

    dataset_truncated = dataset.truncate(after=cutoff.index[0]) #.reset_index(drop=True)

    return dataset_truncated

In [50]:
dataset_truncate

,mm,N
0,0.000000,10.394945
1,0.000000,9.500214
2,0.000049,10.464895
3,0.000098,11.496645
4,0.000146,9.667133
...,...,...
5132,0.427734,1105.925537
5133,0.427832,1105.624023
5134,0.427930,1104.301392
5135,0.428027,1103.967896


In [12]:
def slopefix(dataset,f,g):

    #     #dataset = the specimen data that you want cleaned
    #     #f = startpoint for computing average slope (decimal 0-1 as % of max value)
    #     #g = endpoint for computing average slope (decimal 0-1 as % of max value) 

    slope_array = [] #initialize
    stepsize_array = [] #initialize
    max_val = dataset['N'].max() #initialize
    max_index = dataset.idxmax() #initialize
    y = 10; #interpolation distance, tangent length 
    for i in range(0,max_index.N-y):
        u =dataset.loc[i+y,"N"]-dataset.loc[i,"N"]
        if abs(u)==0:
            u = 0.000001 #remove any calculations that result in zero 

        #take averages of step and slope for data intervals of 'y'
        slope_array.append(u)
        stepsize_array.append(dataset.loc[i+y,"mm"]-dataset.loc[i,"mm"])

    #calculate 50% and 75% of max index load values from interval values 
    stepsize = np.mean(stepsize_array)
    index_f = int(math.ceil(max_index.N*f))
    index_g = int(math.ceil(max_index.N*g))

    slope = np.mean(slope_array[index_f:index_g]) #average slope value between 50% and 75% of the max load

    ind = math.floor(dataset.N[index_g]/slope) #how many increments of the average slope until 75% max is reached

    replacement_load = pd.Series([0]*ind,dtype='float') #initialize
    replacement_disp = pd.Series([0]*ind,dtype='float') #initialize

    #linear increments of averaged slope and step size
    for i in range(1,ind):
        replacement_load[i] = slope*i
        replacement_disp[i] = stepsize*i 

    #add back the replacement load and replacement displacement 

    replacement_df = {'mm':replacement_disp,'N':replacement_load}
    replacement_df = pd.DataFrame(replacement_df)

    #need to rewrite the step size on displacement, for-loop subtracting first index
    dataset_truncate = dataset.truncate(before=index_g).reset_index(drop=True)
    j = dataset_truncate.mm[0]-replacement_disp[ind-1]

    for i in range(0,len(dataset_truncate.mm)):
        dataset_truncate.mm[i] = dataset_truncate.mm[i]-j

    #final data set 
    dataset_new = pd.concat([replacement_df,dataset_truncate], ignore_index=True)

    return dataset_new

In [13]:
# DB_N_5_new.plot(x='mm',y='N')